# Missing data

## Step 1: Determine type of missing data

Ignorable missing data where no remedies are needed: Some data seems to be missing given through the techniques used. As certain features seem to be very specific characteristics, such as for example PoolQC (describing pool quality, which makes only sense if there is a pool), there might be ignorable missing data.

Not_Ignorable missing data, extent and impact have to be assessed, remedies might be needed, if missing data occurs in a random pattern: 
- known. as the dataset has already been pre-cleaned by Kaggle not expected.  
- unknown. Some other data seems to be missing, although a value would be expected. E.g. LotFrontage etc. these missing values cannot be ignored. It has to be analyzed whether a value can be imputed.

As a conclusion there might be ignorable as well as not_ignorable (unknown) missing data in the houseprices dataset. Due to the not_ignorable missing data, the extent and impact of missing data have to be determined. 

### Relevant imports and data load

In [51]:
import pandas as pd

%matplotlib inline

In [86]:
# load data which is stored in the /data folder of the project
train_data = pd.read_csv('../data/train.csv', sep=',', header=0)
test_data = pd.read_csv('../data/test.csv', sep=',', header=0)

In [53]:
train_data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Step 2: Determine extent of missing data

Calculate the following three KPIs
- % and absolute number of missing data overall
- % and absolute number of missing data per feature (variable)
- % and absolute number of missing data per observation

Overall objective is to achieve 0% missing data, as algorithms/statistics cannot deal with missing values.

In [54]:
# overall missing data
def missing_data_overall(df):
    overall_missing = df.isnull().sum().sum()
    overall_values = df.shape[0]*df.shape[1]
    missing_perc = overall_missing * 100 / overall_values
    print("Missing values overall: ", overall_missing)
    print("From total values overall: ", overall_values)
    print("Resulting in: {0:.2f}% missing data overall".format(missing_perc))

In [55]:
# missing data per feature
def missing_data_per_feature(df):
    total_features = df.isnull().sum().sort_values(ascending=False)
    percent_features = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
    missing_data_features = pd.concat([total_features, percent_features], axis=1, keys=['TotalMissing', 'Percent'])
    print(missing_data_features.head(30))

In [113]:
# missing data per observation
def missing_data_per_observation(df):
    
    observations_with_missing_data = df.isnull().replace(to_replace=[False, True], value=['','M'])
    
    total_observations = df.isnull().sum(axis=1).sort_values(ascending=False)
    percent_observations = (df.isnull().sum(axis=1)/df.isnull().count(axis=1)*100).sort_values(ascending=False)
    missing_data_observations = pd.concat([total_observations, percent_observations], axis=1, keys=['TotalMissing', 'Percent'])
    
    return missing_data_observations, observations_with_missing_data

In [57]:
missing_data_overall(train_data)

Missing values overall:  6965
From total values overall:  118260
Resulting in: 5.89% missing data overall


5.89% doesn't seem too high (e.g. such as for instance >50% of missing data) and missing data should be feasible to be remediated without biasing results.  

Proceed with analysis per feature and per observation.

In [58]:
missing_data_per_feature(train_data)

              TotalMissing    Percent
PoolQC                1453  99.520548
MiscFeature           1406  96.301370
Alley                 1369  93.767123
Fence                 1179  80.753425
FireplaceQu            690  47.260274
LotFrontage            259  17.739726
GarageCond              81   5.547945
GarageType              81   5.547945
GarageYrBlt             81   5.547945
GarageFinish            81   5.547945
GarageQual              81   5.547945
BsmtExposure            38   2.602740
BsmtFinType2            38   2.602740
BsmtFinType1            37   2.534247
BsmtCond                37   2.534247
BsmtQual                37   2.534247
MasVnrArea               8   0.547945
MasVnrType               8   0.547945
Electrical               1   0.068493
Utilities                0   0.000000
YearRemodAdd             0   0.000000
MSSubClass               0   0.000000
Foundation               0   0.000000
ExterCond                0   0.000000
ExterQual                0   0.000000
Exterior2nd 

Going through the individual features, there seem to be many ignorable missing data, such as for instance pool quality, alley, fireplace quality etc. in all these cases NA represents the information that the respective building cannot have a pool quality or a fireplace quality etc, because the building has no pool or fireplace

Nevertheless, there are also features with missing values which are not_ignorable and the reason why the values are missing are unkonwn. These are for instance the features LotFrontage or Electrical, as well as MasVnrArea and MasVnrType. 

Let's now 
also have a view on: missing data per observation

In [88]:
missing_data_per_observation, missing_patterns = missing_data_per_observation(train_data)

In [102]:
nr_of_missing_values = missing_data_per_observation.TotalMissing.value_counts()
print_pretty = pd.DataFrame({'nr-of-missing-vals':nr_of_missing_values.index, 'nr-of-observations':nr_of_missing_values.values})
print(print_pretty)

    nr-of-missing-vals  nr-of-observations
0                    4                 635
1                    5                 501
2                    3                 132
3                   10                  60
4                    6                  58
5                    9                  32
6                    2                  16
7                   11                   7
8                    8                   5
9                    7                   5
10                  15                   4
11                  14                   3
12                   1                   2


This table shows that every observation has at least one missing attribute. Therefore, features need to cleaned up first, hoping that overall situation on observations will then improve. 

### Deletion as an option for individual observations or features
Especially for cases where about 50% or even more than 50% of values are missing.

- **PoolQC** will be deleted. Instead Pool y/n will be added. This way, also PoolArea might be deleted in a later step. Depending on correlation to SalePrice.  
- **MiscFeature** will be deleted. It might be replaced with a MiscFeature y/n. Depending on correlation to SalePrice.  
- **Alley** will be deleted. It might be replaced with an Alley y/n. Depending on correlation to SalePrice.  
- **Fence** will be deleted. Might be replaced with Fence y/n. Depending on correlation to SalePrice.
- **FireplaceQu** will be deleted. Instead Fireplace y/n might be added. This way, also NumberOfFireplaces might be replaced with this new added feature. Depending on their correlation to SalePrice.

In [20]:
# drop PoolQC and FireplaceQu. Potential replacement PoolArea and Fireplaces will be decided in a later step
updated_train_data = train_data.drop(['PoolQC', 'FireplaceQu'], axis=1)

In [39]:
# add a new feature HasMiscFeature, HasAlley and HasFence indicating yes=1 or no=0, 
# based upon MiscFeature, Alley, Fence. 

def map_binary (value):
    if value == False:
        return 1
    else:
        return 0

updated_train_data['HasMiscFeature'] = updated_train_data['MiscFeature'].isnull().apply(map_binary)
updated_train_data['HasAlley'] = updated_train_data['Alley'].isnull().apply(map_binary)
updated_train_data['HasFence'] = updated_train_data['Fence'].isnull().apply(map_binary)

In [41]:
# drop MiscFeature, Alley and Fence
updated_train_data = updated_train_data.drop(['MiscFeature', 'Alley', 'Fence'], axis=1)

Let's check the overall status again

In [44]:
missing_data_overall(updated_train_data)

Missing values overall:  868
From total values overall:  115340
Resulting in: 0.75% missing data overall


In [45]:
missing_data_per_feature(updated_train_data)

              TotalMissing    Percent
LotFrontage            259  17.739726
GarageYrBlt             81   5.547945
GarageCond              81   5.547945
GarageType              81   5.547945
GarageFinish            81   5.547945
GarageQual              81   5.547945
BsmtExposure            38   2.602740
BsmtFinType2            38   2.602740
BsmtFinType1            37   2.534247
BsmtCond                37   2.534247
BsmtQual                37   2.534247
MasVnrArea               8   0.547945
MasVnrType               8   0.547945
Electrical               1   0.068493
LandContour              0   0.000000
RoofMatl                 0   0.000000
Exterior1st              0   0.000000
Exterior2nd              0   0.000000
Foundation               0   0.000000
ExterQual                0   0.000000
ExterCond                0   0.000000
Utilities                0   0.000000
MSSubClass               0   0.000000
BsmtFinSF1               0   0.000000
RoofStyle                0   0.000000
YearRemodAdd

In [106]:
missing_data_per_observation, missing_patterns = missing_data_per_observation(updated_train_data)

In [107]:
nr_of_missing_values = missing_data_per_observation.TotalMissing.value_counts()
print_pretty = pd.DataFrame({'nr-of-missing-vals':nr_of_missing_values.index, 'nr-of-observations':nr_of_missing_values.values})
print(print_pretty)

   nr-of-missing-vals  nr-of-observations
0                   0                1094
1                   1                 247
2                   5                  91
3                   6                  13
4                  10                   7
5                   2                   6
6                   3                   2


The minimal sample size came up from 0 before to 1094 now.   
This might be a good starting point and will be saved into a first train data file for future modelling

In [111]:
# delete all observations with missing values. The train_sample_1 will then be stored in the final step of this notebook
train_sample_1 = updated_train_data.dropna(axis=0, how='any')
train_sample_1.shape

(1094, 79)

Now diagnose randomness of missing data in remaining features.

## Step 3: Diagnose randomness of missing data

Diagnosis either via empirical tests or via a visualization of the missing data to check if there are potential patterns. Even though the sample size and number of features are already quite high, the visual approach is used here in a first instance

In [114]:
missing_data_per_observation, missing_patterns = missing_data_per_observation(updated_train_data)

In [115]:
missing_patterns.to_csv("../data/missing_data_patterns.csv", index=False)

In [ ]:
## table with empirical tests to follow.

## Step 4: Remediation of missing data

### Less than 10% missing data for each feature and each observation
NUMERIC data
- analyze if a deletion of the respective features and/or observations would significantly reduce the overall missing data. Check if a collinear feature could take over for the one with missing values. Verify if sample size remains big enough. 
- If not deleted, use a respective imputation method to impute missing values. This should be possible without analyzing possible patterns in the missing data, as with 10% or less missing data, the imputation should not be biased.

NON-NUMERIC data
- add a dummy variable for missing values

### 10% up to 20% missing data for each feature and each observation
NUMERIC data
- analyze if a deletion of the respective features and/or observations would significantly reduce the overall missing data. Check if a collinear feature could take over for the one with missing values. Verify if sample size remains big enough. 
- If not deleted, analyze if there are patterns in the missing data or is the data missed randomly? Based on this outcome use respective MAR methods (patterns found) or respective MCAR (randomly missed data) to impute missing values. T-Test etc. can be used to find out if the data is missed randomly or not. 

NON-NUMERIC data
- add a dummy variable for missing values

###  More than 20% missing data for each feature and each observation
- candidates for deletion. Check if a collinear feature could take over for the one with missing values. Verify if sample size remains big enough. If imputation is really needed, go with regression methods for MCAR and model based techniques for MAR.

### analyze features with less than 20% missing data
MasVnrArea
- option 1) delete 8 observations
- option 2) find imputing values
- option 3) delete feature 

GarageYrBlt
- option 1) possibly correlating with YearBlt, so that GarageYrBlt can be deleted
- option 2) find imputing value. will be a random guess
- option 2) least preferred: delete 81 observations

LotFrontage
- option 1) find a correlating feature, so that LotFrontage can be deleted
- option 2) impute missing values

In [75]:
train_data['LotFrontage'] = train_data['LotFrontage'].fillna(train_data['LotFrontage'].mean())

Assumption that GarageYrBlt is highly correlating to YearBlt. Manual data inspection confirms that picture. But let's compare these two variables. -- comparison of two categorial, ordinal variables. 

In [76]:
train_data = train_data.drop(['GarageYrBlt'], axis=1)

In [77]:
train_data.MasVnrArea.describe()

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

More than 50% of values seem to be on 0. Which is similar to missing in this case?
Checked against the MasVnrType. Same picture here. It seems that also MasVnrType is in more than 50% on None. 
Decision to delete both of these variables

In [78]:
train_data = train_data.drop(['MasVnrType', 'MasVnrArea'], axis=1) 

In [79]:
overall_missing_data(train_data)

Missing values overall:  512
From total values overall:  106580
Resulting in: 0.48% missing data overall


In [80]:
missing_data_per_feature(train_data)

              TotalMissing   Percent
GarageCond              81  5.547945
GarageType              81  5.547945
GarageFinish            81  5.547945
GarageQual              81  5.547945
BsmtFinType2            38  2.602740
BsmtExposure            38  2.602740
BsmtFinType1            37  2.534247
BsmtCond                37  2.534247
BsmtQual                37  2.534247
Electrical               1  0.068493
YearRemodAdd             0  0.000000
RoofStyle                0  0.000000
RoofMatl                 0  0.000000
ExterQual                0  0.000000
Exterior1st              0  0.000000
Exterior2nd              0  0.000000
OverallCond              0  0.000000
ExterCond                0  0.000000
Foundation               0  0.000000
BsmtFinSF1               0  0.000000
YearBuilt                0  0.000000
SalePrice                0  0.000000
OverallQual              0  0.000000
LandContour              0  0.000000
MSSubClass               0  0.000000
MSZoning                 0  0.000000
L

In [81]:
train_data = train_data.drop(train_data.loc[train_data['Electrical'].isnull()].index)

### convert non-numeric features into dummy variables

In [82]:
test_interim = test_data.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu','MasVnrType', 'MasVnrArea', 'GarageYrBlt'], axis=1)

In [83]:
# drop SalePrice and add it after get_dummies
target_variable = train_data['SalePrice']
train_interim = train_data.drop('SalePrice', axis=1)

# concat test and train data. List all train records first, attach the test data second
all_data = pd.concat((train_interim, test_interim), axis=0)

# convert categorical variables into dummy/indicator variable. 
# For missing values an additional column will be created - dummy_na
# The original feature will be dropped - drop_first 
all_dummies = pd.get_dummies(all_data, dummy_na=True, drop_first=True)

# split test and train sets again
train_dummies = all_dummies.iloc[:train_interim.shape[0],:]
test_dummies = all_dummies.iloc[train_interim.shape[0]:,:]

In [85]:
# add SalePrice again
train_dummies = train_dummies.assign(SalePrice=target_variable)

In [ ]:
# store all selected train sample files 
train_sample_1.to_csv("../data/train_sample_1.csv", index=False)